# Waterfall Chart - Sequential Value Changes

**Use Case**: Show cumulative effect of sequential changes (budget variance, sales pipeline, profit analysis)

This notebook demonstrates how to create effective waterfall charts for visualizing how values change step by step from a starting point to an ending point.


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from matplotlib.patches import Rectangle
import warnings
warnings.filterwarnings('ignore')

# Set style for better-looking plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

# Set random seed for reproducibility
np.random.seed(42)

print("Waterfall chart visualization libraries loaded!")


In [ ]:
# Create sample datasets for waterfall charts
# 1. Budget Variance Analysis
budget_variance_data = [
    ('Original Budget', 1000000, 'start'),
    ('Additional Revenue', 150000, 'positive'),
    ('Cost Overruns', -80000, 'negative'),
    ('Savings from Automation', 45000, 'positive'),
    ('Marketing Overspend', -120000, 'negative'),
    ('Bonus Expenses', -60000, 'negative'),
    ('Interest Income', 25000, 'positive'),
    ('Final Budget', None, 'end')  # Will be calculated
]

budget_df = pd.DataFrame(budget_variance_data, columns=['Category', 'Amount', 'Type'])

# 2. Sales Pipeline Analysis
sales_pipeline_data = [
    ('Leads Generated', 10000, 'start'),
    ('Qualified Leads', -6000, 'negative'),  # Lost 6000 unqualified
    ('Proposal Stage', -1500, 'negative'),   # Lost 1500 during qualification
    ('Negotiation Stage', -800, 'negative'), # Lost 800 during proposals
    ('Closed Won', -1200, 'negative'),       # Lost 1200 during negotiation
    ('Final Customers', None, 'end')         # Remaining customers
]

sales_df = pd.DataFrame(sales_pipeline_data, columns=['Stage', 'Change', 'Type'])

# 3. Profit and Loss Waterfall
pnl_data = [
    ('Revenue', 2500000, 'start'),
    ('Cost of Goods Sold', -1200000, 'negative'),
    ('Gross Profit', None, 'intermediate'),
    ('Sales & Marketing', -350000, 'negative'),
    ('R&D Expenses', -200000, 'negative'),
    ('General & Admin', -180000, 'negative'),
    ('EBITDA', None, 'intermediate'),
    ('Depreciation', -120000, 'negative'),
    ('Interest Expense', -80000, 'negative'),
    ('Taxes', -105000, 'negative'),
    ('Net Income', None, 'end')
]

pnl_df = pd.DataFrame(pnl_data, columns=['Category', 'Amount', 'Type'])

# 4. Project Resource Allocation
project_resources_data = [
    ('Initial Team', 100, 'start'),
    ('New Hires Q1', 15, 'positive'),
    ('Departures Q1', -8, 'negative'),
    ('New Hires Q2', 25, 'positive'),
    ('Departures Q2', -12, 'negative'),
    ('Contractor Addition', 20, 'positive'),
    ('Project Completion', -30, 'negative'),
    ('Final Team Size', None, 'end')
]

resources_df = pd.DataFrame(project_resources_data, columns=['Event', 'Change', 'Type'])

# 5. Customer Retention Analysis
retention_data = [
    ('Starting Customers', 5000, 'start'),
    ('New Acquisitions', 800, 'positive'),
    ('Voluntary Churn', -450, 'negative'),
    ('Involuntary Churn', -200, 'negative'),
    ('Win-back Campaign', 150, 'positive'),
    ('Seasonal Loss', -300, 'negative'),
    ('Referral Program', 120, 'positive'),
    ('Ending Customers', None, 'end')
]

retention_df = pd.DataFrame(retention_data, columns=['Activity', 'Change', 'Type'])

print("Sample waterfall chart datasets created:")
print(f"Budget Variance: {len(budget_df)} categories")
print(f"Sales Pipeline: {len(sales_df)} stages")
print(f"P&L Analysis: {len(pnl_df)} line items")
print(f"Resource Allocation: {len(resources_df)} events")
print(f"Customer Retention: {len(retention_df)} activities")


In [ ]:
# Function to create waterfall charts
def create_waterfall_chart(df, category_col, amount_col, type_col, title, ax, 
                          start_color='blue', positive_color='green', negative_color='red', 
                          end_color='navy'):
    """Create a waterfall chart"""
    
    # Calculate cumulative values
    cumulative = []
    running_total = 0
    
    for i, row in df.iterrows():
        if row[type_col] == 'start':
            running_total = row[amount_col]
            cumulative.append(running_total)
        elif row[type_col] == 'end':
            cumulative.append(running_total)
        elif row[type_col] == 'intermediate':
            cumulative.append(running_total)
        else:
            cumulative.append(running_total + row[amount_col])
            running_total += row[amount_col]
    
    # Update end values
    df_copy = df.copy()
    for i, row in df_copy.iterrows():
        if row[type_col] in ['end', 'intermediate']:
            df_copy.loc[i, amount_col] = cumulative[i]
    
    # Create bars
    x_pos = range(len(df_copy))
    
    for i, (idx, row) in enumerate(df_copy.iterrows()):
        if row[type_col] == 'start':
            # Start bar from 0
            ax.bar(i, row[amount_col], color=start_color, alpha=0.8, 
                  label='Start' if i == 0 else "")
            height = row[amount_col]
            bottom = 0
        elif row[type_col] == 'end' or row[type_col] == 'intermediate':
            # End/intermediate bar from 0
            ax.bar(i, row[amount_col], color=end_color, alpha=0.8,
                  label='End/Total' if row[type_col] == 'end' and 'End' not in [l.get_label() for l in ax.get_legend_handles_labels()[0]] else "")
            height = row[amount_col]
            bottom = 0
        else:
            # Floating bars for changes
            if row[amount_col] > 0:
                color = positive_color
                label = 'Positive' if 'Positive' not in [l.get_label() for l in ax.get_legend_handles_labels()[0]] else ""
            else:
                color = negative_color
                label = 'Negative' if 'Negative' not in [l.get_label() for l in ax.get_legend_handles_labels()[0]] else ""
            
            # Calculate bottom position (previous cumulative)
            bottom = cumulative[i-1] if i > 0 else 0
            if row[amount_col] < 0:
                bottom = cumulative[i]  # For negative, start from the new position
            
            ax.bar(i, abs(row[amount_col]), bottom=bottom, color=color, alpha=0.8, label=label)
            height = abs(row[amount_col])
        
        # Add value labels
        if row[amount_col] is not None:
            if row[type_col] in ['start', 'end', 'intermediate']:
                label_y = height / 2
                ax.text(i, label_y, f'{row[amount_col]:,.0f}', 
                       ha='center', va='center', fontweight='bold', fontsize=9)
            else:
                if row[amount_col] > 0:
                    label_y = bottom + height / 2
                else:
                    label_y = bottom + height / 2
                ax.text(i, label_y, f'{row[amount_col]:+,.0f}', 
                       ha='center', va='center', fontweight='bold', fontsize=9)
    
    # Connect bars with lines
    for i in range(len(df_copy) - 1):
        current_end = cumulative[i]
        next_start = cumulative[i] if df_copy.iloc[i+1][type_col] not in ['start'] else 0
        ax.plot([i + 0.4, i + 0.6], [current_end, current_end], 'k--', alpha=0.5)
    
    # Formatting
    ax.set_xlabel('Categories')
    ax.set_ylabel('Amount')
    ax.set_title(title, fontweight='bold', pad=20)
    ax.set_xticks(x_pos)
    ax.set_xticklabels([row[category_col] for _, row in df_copy.iterrows()], rotation=45, ha='right')
    
    # Add horizontal line at zero
    ax.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    
    return ax

# Create waterfall charts
fig, axes = plt.subplots(2, 2, figsize=(20, 16))
fig.suptitle('Waterfall Chart Visualizations - Sequential Changes', fontsize=16, fontweight='bold')

# 1. Budget Variance Waterfall
ax1 = axes[0, 0]
create_waterfall_chart(budget_df, 'Category', 'Amount', 'Type', 
                      'Budget Variance Analysis', ax1)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# 2. Sales Pipeline Waterfall  
ax2 = axes[0, 1]
create_waterfall_chart(sales_df, 'Stage', 'Change', 'Type',
                      'Sales Pipeline Conversion', ax2,
                      start_color='lightblue', negative_color='orange')

# 3. P&L Waterfall (simplified version)
ax3 = axes[1, 0]
# Create simplified P&L for visualization
pnl_simple = pd.DataFrame([
    ('Revenue', 2500000, 'start'),
    ('COGS', -1200000, 'negative'),
    ('OpEx', -730000, 'negative'),
    ('EBITDA', None, 'intermediate'),
    ('D&A + Interest', -200000, 'negative'),
    ('Taxes', -105000, 'negative'),
    ('Net Income', None, 'end')
], columns=['Category', 'Amount', 'Type'])

create_waterfall_chart(pnl_simple, 'Category', 'Amount', 'Type',
                      'Profit & Loss Waterfall', ax3)
ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000000:.1f}M'))

# 4. Resource Allocation Waterfall
ax4 = axes[1, 1]
create_waterfall_chart(resources_df, 'Event', 'Change', 'Type',
                      'Project Team Changes', ax4,
                      start_color='purple', positive_color='darkgreen')

plt.tight_layout()
plt.show()


In [ ]:
# Advanced waterfall chart techniques
fig, axes = plt.subplots(2, 2, figsize=(20, 16))
fig.suptitle('Advanced Waterfall Chart Techniques', fontsize=16, fontweight='bold')

# 1. Multi-period Budget Analysis
ax1 = axes[0, 0]

# Create quarterly budget progression
quarters = ['Q1 Start', 'Q1 Revenue', 'Q1 Costs', 'Q1 End', 
           'Q2 Revenue', 'Q2 Costs', 'Q2 End',
           'Q3 Revenue', 'Q3 Costs', 'Q3 End',
           'Q4 Revenue', 'Q4 Costs', 'Year End']

quarterly_data = [
    ('Q1 Start', 1000000, 'start'),
    ('Q1 Revenue', 250000, 'positive'),
    ('Q1 Costs', -180000, 'negative'),
    ('Q1 End', None, 'intermediate'),
    ('Q2 Revenue', 280000, 'positive'),
    ('Q2 Costs', -200000, 'negative'),
    ('Q2 End', None, 'intermediate'),
    ('Q3 Revenue', 320000, 'positive'),
    ('Q3 Costs', -220000, 'negative'),
    ('Q3 End', None, 'intermediate'),
    ('Q4 Revenue', 350000, 'positive'),
    ('Q4 Costs', -250000, 'negative'),
    ('Year End', None, 'end')
]

quarterly_df = pd.DataFrame(quarterly_data, columns=['Period', 'Amount', 'Type'])
create_waterfall_chart(quarterly_df, 'Period', 'Amount', 'Type',
                      'Quarterly Budget Progression', ax1)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# Add quarterly separators
for i in [3, 6, 9]:
    ax1.axvline(x=i+0.5, color='gray', linestyle='--', alpha=0.5)

# 2. Customer Retention with Cohort Analysis
ax2 = axes[0, 1]

# Enhanced retention analysis
retention_enhanced = pd.DataFrame([
    ('Jan Start', 5000, 'start'),
    ('New Signups', 400, 'positive'),
    ('Trial Dropouts', -280, 'negative'),
    ('Feb Total', None, 'intermediate'),
    ('New Signups', 450, 'positive'),
    ('Churn', -350, 'negative'),
    ('Mar Total', None, 'intermediate'),
    ('New Signups', 380, 'positive'),
    ('Churn', -320, 'negative'),
    ('Q1 End', None, 'end')
], columns=['Period', 'Change', 'Type'])

create_waterfall_chart(retention_enhanced, 'Period', 'Change', 'Type',
                      'Customer Retention - Q1 Analysis', ax2,
                      start_color='darkblue', end_color='darkgreen')

# 3. Variance Analysis with Targets
ax3 = axes[1, 0]

# Performance vs targets
variance_data = pd.DataFrame([
    ('Target Revenue', 1000000, 'start'),
    ('Volume Variance', 50000, 'positive'),
    ('Price Variance', -30000, 'negative'),
    ('Mix Variance', 20000, 'positive'),
    ('Actual Revenue', None, 'intermediate'),
    ('Target Costs', -600000, 'negative'),
    ('Efficiency Gains', 40000, 'positive'),
    ('Inflation Impact', -25000, 'negative'),  
    ('Actual Profit', None, 'end')
], columns=['Component', 'Variance', 'Type'])

# Custom colors for variance analysis
create_waterfall_chart(variance_data, 'Component', 'Variance', 'Type',
                      'Revenue & Cost Variance Analysis', ax3,
                      start_color='gold', positive_color='darkgreen', 
                      negative_color='darkred', end_color='darkblue')
ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# 4. Cash Flow Waterfall
ax4 = axes[1, 1]

cash_flow_data = pd.DataFrame([
    ('Opening Cash', 500000, 'start'),
    ('Operations', 200000, 'positive'),
    ('Investments', -150000, 'negative'),
    ('Financing', 100000, 'positive'),
    ('Dividends', -80000, 'negative'),
    ('FX Impact', -20000, 'negative'),
    ('Closing Cash', None, 'end')
], columns=['Activity', 'Amount', 'Type'])

create_waterfall_chart(cash_flow_data, 'Activity', 'Amount', 'Type',
                      'Cash Flow Statement', ax4,
                      start_color='lightgreen', end_color='darkgreen')
ax4.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

plt.tight_layout()
plt.show()


In [ ]:
# Interactive waterfall chart example (Plotly structure)
print("Interactive Waterfall Charts (Plotly):")
print("=" * 50)

print("\n1. Interactive Budget Waterfall")
print("Code structure:")
print("""
# Prepare data for Plotly waterfall
categories = budget_df['Category'].tolist()
amounts = budget_df['Amount'].fillna(0).tolist()
types = budget_df['Type'].tolist()

# Create measure array for Plotly
measure = []
for t in types:
    if t == 'start':
        measure.append('absolute')
    elif t == 'end':
        measure.append('total')
    else:
        measure.append('relative')

fig = go.Figure(go.Waterfall(
    name="Budget Analysis",
    orientation="v",
    measure=measure,
    x=categories,
    y=amounts,
    connector={"line":{"color":"rgb(63, 63, 63)"}},
    increasing={"marker":{"color":"green"}},
    decreasing={"marker":{"color":"red"}},
    totals={"marker":{"color":"blue"}}
))

fig.update_layout(title="Interactive Budget Variance Waterfall")
fig.show()
""")

print("\n2. Multi-series Waterfall Comparison")
print("Code structure:")
print("""
# Compare actual vs budget waterfalls side by side
fig = make_subplots(rows=1, cols=2, 
                   subplot_titles=('Actual Performance', 'Budget Plan'))

# Add actual data
fig.add_trace(go.Waterfall(...), row=1, col=1)
# Add budget data  
fig.add_trace(go.Waterfall(...), row=1, col=2)

fig.show()
""")


In [ ]:
# Statistical analysis of waterfall data
print("Waterfall Chart Statistical Analysis:")
print("=" * 50)

# 1. Budget Variance Analysis
print("1. BUDGET VARIANCE BREAKDOWN:")

# Calculate actual values for analysis
budget_analysis = budget_df.copy()
running_total = 0
actual_amounts = []

for _, row in budget_analysis.iterrows():
    if row['Type'] == 'start':
        running_total = row['Amount']
        actual_amounts.append(running_total)
    elif row['Type'] == 'end':
        actual_amounts.append(running_total)
    else:
        if row['Amount'] is not None:
            running_total += row['Amount']
        actual_amounts.append(running_total if row['Type'] != 'intermediate' else running_total)

budget_analysis['Actual_Amount'] = actual_amounts

original_budget = budget_analysis[budget_analysis['Type'] == 'start']['Amount'].iloc[0]
final_budget = actual_amounts[-1]
total_variance = final_budget - original_budget

print(f"   Original Budget: ${original_budget:,}")
print(f"   Final Budget: ${final_budget:,}")
print(f"   Total Variance: ${total_variance:+,} ({(total_variance/original_budget)*100:+.1f}%)")

# Analyze positive vs negative impacts
positive_changes = budget_analysis[(budget_analysis['Type'] == 'positive') & (budget_analysis['Amount'] > 0)]
negative_changes = budget_analysis[(budget_analysis['Type'] == 'negative') & (budget_analysis['Amount'] < 0)]

total_positive = positive_changes['Amount'].sum()
total_negative = negative_changes['Amount'].sum()

print(f"\n   Positive Impacts: ${total_positive:+,}")
for _, change in positive_changes.iterrows():
    contribution = (change['Amount'] / total_positive) * 100
    print(f"     {change['Category']}: ${change['Amount']:+,} ({contribution:.1f}%)")

print(f"\n   Negative Impacts: ${total_negative:+,}")
for _, change in negative_changes.iterrows():
    contribution = (abs(change['Amount']) / abs(total_negative)) * 100
    print(f"     {change['Category']}: ${change['Amount']:+,} ({contribution:.1f}%)")

# 2. Sales Pipeline Analysis
print(f"\n2. SALES CONVERSION ANALYSIS:")

sales_analysis = sales_df.copy()
running_count = 0
conversion_rates = []

for i, row in sales_analysis.iterrows():
    if row['Type'] == 'start':
        running_count = row['Change']
        initial_leads = running_count
    elif row['Type'] == 'end':
        final_customers = running_count
    else:
        if row['Change'] is not None:
            prev_count = running_count
            running_count += row['Change']  # Change is negative for losses
            if prev_count > 0:
                conversion_rate = (abs(row['Change']) / prev_count) * 100
                conversion_rates.append((row['Stage'], conversion_rate, running_count))

print(f"   Initial Leads: {initial_leads:,}")
print(f"   Final Customers: {final_customers:,}")
print(f"   Overall Conversion Rate: {(final_customers/initial_leads)*100:.2f}%")

print(f"\n   Stage-by-Stage Conversion:")
for stage, rate, remaining in conversion_rates:
    print(f"     {stage}: {rate:.1f}% loss rate, {remaining:,} remaining")

# 3. Resource Utilization Analysis
print(f"\n3. RESOURCE ALLOCATION EFFICIENCY:")

resources_analysis = resources_df.copy()
initial_team = resources_analysis[resources_analysis['Type'] == 'start']['Change'].iloc[0]

# Calculate team changes
additions = resources_analysis[resources_analysis['Change'] > 0]['Change'].sum() - initial_team
departures = abs(resources_analysis[resources_analysis['Change'] < 0]['Change'].sum())
net_change = additions - departures

print(f"   Initial Team Size: {initial_team}")
print(f"   Total Additions: {additions}")
print(f"   Total Departures: {departures}")
print(f"   Net Change: {net_change:+}")
print(f"   Turnover Rate: {(departures/(initial_team + additions))*100:.1f}%")

# Calculate retention rate
if additions > 0:
    retention_rate = ((initial_team + additions - departures) / (initial_team + additions)) * 100
    print(f"   Retention Rate: {retention_rate:.1f}%")

# 4. Customer Retention Metrics
print(f"\n4. CUSTOMER RETENTION INSIGHTS:")

retention_analysis = retention_df.copy()
starting_customers = retention_analysis[retention_analysis['Type'] == 'start']['Change'].iloc[0]

# Calculate churn and acquisition
new_acquisitions = retention_analysis[retention_analysis['Change'] > 0]['Change'].sum() - starting_customers
total_churn = abs(retention_analysis[retention_analysis['Change'] < 0]['Change'].sum())

# Calculate final customer count
final_customers = starting_customers + retention_analysis['Change'].fillna(0).sum()

print(f"   Starting Customers: {starting_customers:,}")
print(f"   New Acquisitions: {new_acquisitions:,}")
print(f"   Total Churn: {total_churn:,}")
print(f"   Ending Customers: {final_customers:,}")

# Calculate rates
churn_rate = (total_churn / starting_customers) * 100
acquisition_rate = (new_acquisitions / starting_customers) * 100
net_growth_rate = ((final_customers - starting_customers) / starting_customers) * 100

print(f"\n   Churn Rate: {churn_rate:.2f}%")
print(f"   Acquisition Rate: {acquisition_rate:.2f}%")
print(f"   Net Growth Rate: {net_growth_rate:+.2f}%")

# Break down churn by type
voluntary_churn = abs(retention_analysis[retention_analysis['Activity'] == 'Voluntary Churn']['Change'].iloc[0])
involuntary_churn = abs(retention_analysis[retention_analysis['Activity'] == 'Involuntary Churn']['Change'].iloc[0])

print(f"   Voluntary Churn: {voluntary_churn} ({(voluntary_churn/total_churn)*100:.1f}% of total churn)")
print(f"   Involuntary Churn: {involuntary_churn} ({(involuntary_churn/total_churn)*100:.1f}% of total churn)")

# 5. Waterfall Chart Design Effectiveness
print(f"\n5. WATERFALL CHART DESIGN ANALYSIS:")

datasets = {
    'Budget Variance': budget_df,
    'Sales Pipeline': sales_df,
    'Resource Changes': resources_df,
    'Customer Retention': retention_df
}

print(f"   Chart Complexity Analysis:")
for name, df in datasets.items():
    n_steps = len(df) - 2  # Exclude start and end
    
    # Calculate value ranges
    numeric_values = df[df.columns[1]].dropna()
    if len(numeric_values) > 0:
        value_range = numeric_values.max() - numeric_values.min()
        max_abs_value = max(abs(numeric_values.max()), abs(numeric_values.min()))
        
        # Assess visual complexity
        if n_steps <= 5 and value_range <= max_abs_value * 2:
            complexity = "Simple"
        elif n_steps <= 8 and value_range <= max_abs_value * 5:
            complexity = "Moderate"
        else:
            complexity = "Complex"
        
        print(f"     {name}: {n_steps} steps, {complexity} visualization")

print(f"\nWaterfall Chart Best Practices:")
print("✓ Start with a clear baseline (starting value)")
print("✓ Use consistent colors (green=positive, red=negative)")
print("✓ Show connecting lines between bars")
print("✓ Label all values clearly")
print("✓ Include intermediate totals for complex flows")
print("✓ Limit to 10-12 steps for readability")
print("✓ Use appropriate scale to show all changes clearly")

print(f"\nWhen to Use Waterfall Charts:")
print("• Budget variance analysis")
print("• Sales funnel conversion tracking")
print("• P&L statement visualization")
print("• Cash flow analysis")
print("• Customer acquisition/retention flows")
print("• Project resource allocation changes")
print("• Performance variance explanations")

print(f"\nAlternatives to Consider:")
print("• Stacked bar charts for simple before/after")
print("• Sankey diagrams for complex flows")
print("• Line charts for time series changes")
print("• Bridge charts for variance analysis")
